<a href="https://colab.research.google.com/github/tmathema/myrepo/blob/master/Data_extraction_dissertation_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Extraction**

In this note book l will describe how the data extraction processes l embarked on, to prepare the data for analysis.  

The data consists of UCT publications from the (https://open.uct.ac.za/oai/request?verb=ListSets).
The aim of this process was to extract publications content which was to be used for analysis. This meant that we would want to optimally extract paragraph texts and subscripts e.g. footnotes, html links or any other texts that would distort the data. 

This process was adapted using this towards-data-science article ( 
https://towardsdatascience.com/extracting-headers-and-paragraphs-from-pdf-using-pymupdf-676e8421c467)

The text was extracted on the basis that components of publication can be  differentiated by font size and weight. For example, headers may have the largest fonts and subscripts may have the smallest fonts in the publication. As such, the most used font type in the publication is definitely used in the paragraph. Using this methodology, the paragraph texts were identified by the p tag. The following code shows this implementation. 



In [ ]:
#connecting to google drive in order to access the data.
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
#installing PyMuPDF
!pip install PyMuPDF

     |████████████████████████████████| 8.8 MB 4.6 MB/s 


In [ ]:
pip install flashtext

In [ ]:
#installing textacy
!pip install textacy

In [ ]:
#import os
#os.listdir('/content/gdrive/MyDrive/data')
#import glob
#glob.glob('/content/gdrive/MyDrive/data/*')

In [ ]:
import fitz
import glob
papers = []
nonpapers = []
path =  '/content/gdrive/MyDrive/data_uct/*'
files = glob.glob(path)
for file in files:
  try:
    doc = fitz.open(file)
    papers.append(doc)
  except: 
    nonpapers.append(file)
    continue

mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no objects found
mupdf: cannot recognize version marker
mupdf: no object

In [ ]:
nonpapers

['/content/gdrive/MyDrive/data_uct/pdf_1578.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_4013.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3667.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_2143.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3424.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3437.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_1870.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3525.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3511.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_2296.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3324.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_2974.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_2178.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_2443.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3835.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_1577.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3075.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3666.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3991.pdf',
 '/content/gdrive/MyDrive/data_uct/pdf_3822.pdf',


In [ ]:
len(papers)

4946

In [ ]:
papers

[Document('/content/gdrive/MyDrive/data_uct/pdf_726.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_2877.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_944.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_725.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_1432.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_2595.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_2193.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_1627.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_2297.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_735.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_2127.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_2709.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_973.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_778.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_692.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_743.pdf'),
 Document('/content/gdrive/MyDrive/data_uct/pdf_

First we created a dictionary with all the different font types and attributes and a list with font types and the frequencies it was used in a document.

In [ ]:
 # function for counting font sizes and font types used in the publications 
 
 from operator import itemgetter
 def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    
    styles = {}
    font_counts = {}
    for page in doc:
        blocks = page.getText("dict")["blocks"] 
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}
                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage
    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)
    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")
    return font_counts, styles

In [ ]:
# the fonts output
font_counts = []
nonfonts = []
for doc in papers:
  try:
    font = fonts(doc, granularity=False)
    font_counts.append(font)
  except:
    nonfonts.append(doc)

In [ ]:
font_counts[0]

In [ ]:
nonfonts

In [ ]:
papers_filtered = []
for elem in papers:
  if elem not in nonfonts:
    papers_filtered.append(elem)


In [ ]:
len(papers_filtered)

4929

In [ ]:
papers_filtered

In [ ]:
#head
font_counts[:2]

Then we assigned tags to each font size to identify paragraphs and other components of the publications

In [ ]:
# function to signing tags to the font sizes 
def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size
    # sorting the font sizes high to low, so that we can append the right integer to each tag 
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)
    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)
    return size_tag

In [ ]:
#fiting the font tags function to the data
size_tags = []
styles = [sub_list[1] for sub_list in font_counts]
font = [sub_list[0] for sub_list in font_counts]
for (i,j) in zip(font, styles):
  size_tag = font_tags(i,j) 
  size_tags.append(size_tag)

In [ ]:
#head
size_tags[:2]

We then iterate over the pages of the publications to indentify parapraphs and other components.

In [ ]:
# function to assign tags to the text in the data
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span
    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text
                # REMEMBER: multiple fonts and sizes are possible IN one block
                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:
                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']
                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']
                                previous_s = s
                    # new block started, indicating with a pipe

                    block_string += ""
                header_para.append(block_string)
    return header_para

In [ ]:
# fitting the size tag function to the data.
non_elements = []
elem = []
for doc, size in zip(papers_filtered, size_tags):
  try:
    elements = headers_para(doc, size)
    elem.append(elements)
  except:
    non_elements.append(doc)

In [ ]:
#example of one document
elem[5]

In [ ]:
non_elements

[]

In [ ]:
#len(elem)

In [ ]:
# extracting paragraphs form the publications and uploading the to a list
import re
paper_text_p = ""
pattern_p = re.compile(r'<p>')
paper_p = []
for doc in elem:
  for line in doc:
      if pattern_p.findall(line):
        paper_text_p += line
  paper_p.append(paper_text_p) 
  paper_text_p =""

In [ ]:
import re
paper_text_p = ""
pattern_p = re.compile(r'<h1>')
paper_h = []
for doc in elem:
  for line in doc:
      if pattern_p.findall(line):
        paper_text_p += line
  paper_h.append(paper_text_p) 
  paper_text_p =""

In [ ]:
paper_h

In [ ]:
len(paper_h)

4929

**Data cleaning**
This section will cover datacleaning processes done on the data.

In [ ]:
len(paper_p)

4929

In [ ]:
paper_p[123]

'<p>Eliada Griffin-EL <p>Graduate School of Business, University of Cape Town <p>Accepted: January 2014 <p>JEL: D85, J15, L26, M13 <p>Micro-sized but formally registered businesses  of the South African entrepreneurial landscape  occupy a particular place on the country’s  development agenda. As micro-enterprises,  they face the well-known challenges of  achieving scale and productivity levels; yet, as  formally registered businesses, they serve as a  bridge between the central and peripheral  markets that both constitute the South African  economy. With their failure rates presumed to  be high, yet their success presumed to be  important, understanding how they facilitate  their entrepreneurial process while having  limited capacity becomes a question of both  intrigue and development implications for  economic inclusivity.  <p>The purpose of this study is to explore the <p>structural embeddedness of South African  microbusinesses – and to better understand how  they have engaged with

In [ ]:
for i in range(len(paper_p)):
  try:
   paper_p[i] = paper_p[i][paper_p[i].index('<p>1. '):]
  except ValueError:
    pass # or scream: thing not in some_list!
    

In [ ]:
paper_p[123]

'<p>Eliada Griffin-EL <p>Graduate School of Business, University of Cape Town <p>Accepted: January 2014 <p>JEL: D85, J15, L26, M13 <p>Micro-sized but formally registered businesses  of the South African entrepreneurial landscape  occupy a particular place on the country’s  development agenda. As micro-enterprises,  they face the well-known challenges of  achieving scale and productivity levels; yet, as  formally registered businesses, they serve as a  bridge between the central and peripheral  markets that both constitute the South African  economy. With their failure rates presumed to  be high, yet their success presumed to be  important, understanding how they facilitate  their entrepreneurial process while having  limited capacity becomes a question of both  intrigue and development implications for  economic inclusivity.  <p>The purpose of this study is to explore the <p>structural embeddedness of South African  microbusinesses – and to better understand how  they have engaged with

In [ ]:
# removing the <p> tags
paper_p_clean = [re.sub('<p>', ' ',text) for text in paper_p]

In [ ]:
paper_h_clean = [re.sub('<h1>', ' ',text) for text in paper_h]

In [ ]:
paper_h_clean

In [ ]:
#paper_p_clean = [re.sub(' fi ', 'fi',text) for text in paper_p_clean]
#paper_p_clean = [re.sub(' fa ', 'fa',text) for text in paper_p_clean]
#paper_p_clean = [re.sub(' fe ', 'fe',text) for text in paper_p_clean]
#paper_p_clean = [re.sub(' fo ', 'fo',text) for text in paper_p_clean]
#paper_p_clean = [re.sub(' fu ', 'fu',text) for text in paper_p_clean]
#paper_p_clean = [re.sub(' fl ', 'fl',text) for text in paper_p_clean]

In [ ]:
paper_p_clean[23]

' J. Adam  et al. (ALICE Collaboration) The properties of an extreme state of matter, the quark-gluon plasma (QGP), are studied by colliding heavy ions at BNL ’ s Relativistic Heavy Ion Collider (RHIC) and at CERN ’ s Large Hadron Collider (LHC). One of the most widely utilized physical phenomena in the exploration of QGP properties is collective anisotropic flow  [1,2] . The large elliptic flow discovered at RHIC energies  [3] , which at the LHC energy of 2.76 TeV is 30% larger  [4]  and is recently reported in Ref.  [5]  to increase even further at 5.02 TeV, demonstrated that the QGP behaves like a strongly coupled liquid with a very small ratio of the shear viscosity to entropy density,  η =s , which is close to a universal lower bound of  1 = 4 π  [6] . Anisotropic flow is traditionally quantified with harmonics  v  and corresponding symmetry plane angles ψ  in the Fourier series decomposition of the particle azimuthal distribution (parametrized with azimuthal angle φ ) in the plan

In [ ]:
#removing the '-' contiuation character
paper_p_clean = [re.sub('\-\s', '', text) for text in paper_p_clean]

In [ ]:
paper_p_clean[23]

' J. Adam  et al. (ALICE Collaboration) The properties of an extreme state of matter, the quark-gluon plasma (QGP), are studied by colliding heavy ions at BNL ’ s Relativistic Heavy Ion Collider (RHIC) and at CERN ’ s Large Hadron Collider (LHC). One of the most widely utilized physical phenomena in the exploration of QGP properties is collective anisotropic flow  [1,2] . The large elliptic flow discovered at RHIC energies  [3] , which at the LHC energy of 2.76 TeV is 30% larger  [4]  and is recently reported in Ref.  [5]  to increase even further at 5.02 TeV, demonstrated that the QGP behaves like a strongly coupled liquid with a very small ratio of the shear viscosity to entropy density,  η =s , which is close to a universal lower bound of  1 = 4 π  [6] . Anisotropic flow is traditionally quantified with harmonics  v  and corresponding symmetry plane angles ψ  in the Fourier series decomposition of the particle azimuthal distribution (parametrized with azimuthal angle φ ) in the plan

In [ ]:
#replacing urls
#from textacy import preprocessing
#for i in range(len(paper_p)):
#  paper_p_clean[i] = preprocessing.replace_urls(paper_p_clean[i],replace_with='')

In [ ]:
paper_p_clean[16]

' M. Aaboud  et al. (ATLAS Collaboration) I. INTRODUCTION Events with an energetic jet and large missing transverse momentum  ~p (with magnitude  E ) in the final state constitute a clean and distinctive signature in searches for new physics beyond the Standard Model (SM) at colliders. Such signatures are referred to as monojetlike in this paper. In particular, monojet (as well as monophoton and monoW=Z ) final states have been studied at the Large Hadron Collider (LHC)  [1 – 15]  in the context of searches for large extra spatial dimensions (LED), supersymmetry (SUSY), and weakly interacting massive particles (WIMPs) as candidates for dark matter. The Arkani-Hamed, Dimopoulos, and Dvali (ADD) model for LED  [16]  explains the large difference between the electroweak unification scale at  O ð 10 Þ  GeV and the Planck scale  M  ∼  O ð 10 Þ  GeV by postulating the presence of  n  extra spatial dimensions of size  R , and defining a fundamental Planck scale in  4  þ  n  dimensions,  M , g

In [ ]:
#converting to a dataframe
from pandas import DataFrame
papers_title = DataFrame(paper_h_clean, columns=['titles_h1'])
papers_df = DataFrame(paper_p_clean, columns=['text'])
papers_filtered = DataFrame(papers_filtered, columns=['pdfs'])

In [ ]:
papers_title['titles_h1'] = papers_title['titles_h1'].str.replace(r"^'\\", '')

In [ ]:
papers_title['titles_h1'] 

0        Mind the gaps: investigating the cause of the...
1        Complex statistics and diffusion in nonlinear...
2        Characteristics and outcome of children with ...
3        Trauma care and capture rate of variables of ...
4        Tracking the Money for Open Educational Resou...
                              ...                        
4924                                                     
4925     LANGUAGE AND ACADEMIC ACHIEVEMENT:  PERSPECTI...
4926     Workers’ Education and Political consciousnes...
4927                                                     
4928     The role of postgraduate students in co-autho...
Name: titles_h1, Length: 4929, dtype: object

In [ ]:
import pandas as pd
papers_df = pd.concat([papers_title, papers_filtered, papers_df], axis =1)

In [ ]:
papers_df.shape

(4929, 3)

In [ ]:
papers_df.iloc[0]

titles_h1     Mind the gaps: investigating the cause of the...
pdfs         (page 0 of /content/gdrive/MyDrive/data_uct/pd...
text          Low-lying areas of the Cape at Africa’s south...
Name: 0, dtype: object

In [ ]:
papers_df.index

RangeIndex(start=0, stop=4929, step=1)

In [ ]:
papers_df.pdfs.to_csv('/content/gdrive/MyDrive/paper_nonerrors.csv')

In [ ]:
import numpy as np
for col in papers_df.columns:
    if papers_df[col].dtype==object:
        papers_df[col]=papers_df[col].apply(lambda x: np.nan if x==np.nan else str(x).encode('utf-8', 'replace').decode('utf-8'))

In [ ]:
papers_df.to_csv('/content/gdrive/MyDrive/papers_df.csv')

In [ ]:
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/uct1.csv")

In [ ]:
data.head(1)

In [ ]:
common_df = pd.merge(data, papers_df, left_on='title', right_on='titles_h1', how='right')

In [ ]:
common_df.shape

(4929, 13)

In [ ]:
common_df.head()

,Unnamed: 0,URL,title,Author_,Date_,Description,Source_,Source_1,PDF_URL,PDF_text_,titles_h1,pdfs,text
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mind the gaps: investigating the cause of the...,(page 0 of /content/gdrive/MyDrive/data_uct/pd...,Low-lying areas of the Cape at Africa’s south...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complex statistics and diffusion in nonlinear...,(page 0 of /content/gdrive/MyDrive/data_uct/pd...,"Ch. G. Antonopoulos, T. Bountis, Ch. Skokos, ..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Characteristics and outcome of children with ...,(page 0 of /content/gdrive/MyDrive/data_uct/pd...,Juvenile dermatomyositis (JDM) is an immune m...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Characteristics and outcome of children with ...,(page 0 of /content/gdrive/MyDrive/data_uct/pd...,Juvenile dermatomyositis (JDM) is an immune m...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Trauma care and capture rate of variables of ...,(page 0 of /content/gdrive/MyDrive/data_uct/pd...,Sawe et al. BMC Emergency Medicine ...


In [ ]:
my_list = [726,944,725,1432,2595,2193,1627,2297,735,2127,2709,973,778,692,743,693,690,860,3293,691,689,1855,695,683,1057,686,3898,688,687,685,696,
668,682,724,697,694,678,677,671,679,728,680,3907,669,676,719,667,672,684,956,673,670,656,662,681,666,655,659,
660,
645,
644,
649,
641,
664,
663,
653,
647,
657,
642,
658,
640,
651,
661,
3926,
646,
650,
3897,
643,
639,
2504,
636,
1136,
1504,
3992,
3059,
665,
595,
3220,
638,
652,
1978,
637,
1768,
1754,
596,
3680,
1705,
654,
593,
2329,
2583,
585,
3545,
589,
570,
2960,
1295,
569,
597,
594,
648,
568,
591,
586,
566,
721,
590,
572,
3758,
546,
565,
567,
576,
592,
563,
577,
584,
545,
547,
559,
587,
578,
580,
574,
558,
571,
541,
544,
588,
575,
582,
579,
555,
564,
2274,
561,
539,
859,
557,
542,
718,
556,
2428,
583,
543,
560,
2860,
562,
1046,
2724,
1431,
581,
2876,
2258,
516,
519,
517,
2594,
540,
1056,
3306,
3232,
3292,
1769,
1854,
554,
772,
510,
573,
635,
943,
538,
508,
518,
1135,
514,
515,
723,
505,
520,
521,
522,
509,
1626,
955,
512,
777,
511,
501,
504,
1240,
497,
507,
496,
500,
470,
489,
502,
503,
488,
499,
478,
494,
481,
447,
487,
490,
468,
480,
492,
491,
445,
485,
498,
441,
475,
446,
482,
469,
493,
495,
440,
442,
438,
435,
513,
484,
472,
467,
439,
421,
466,
420,
506,
429,
434,
479,
437,
432,
486,
444,
427,
422,
1446,
419,
464,
2582,
433,
436,
426,
431,
3834,
2782,
476,
428,
430,
2192,
3906,
1977,
443,
2723,
3058,
2442,
477,
3925,
465,
2875,
942,
2708,
3896,
2328,
1503,
1517,
1294,
1767,
3757,
1853,
3679,
1704,
397,
395,
2114,
2142,
3436,
388,
396,
2781,
2126,
392,
389,
390,
393,
3544,
474,
1430,
473,
369,
371,
391,
1753,
387,
386,
381,
3524,
383,
372,
365,
385,
368,
377,
378,
384,
394,
364,
380,
363,
370,
367,
373,
362,
355,
350,
360,
374,
379,
376,
341,
366,
382,
348,
338,
346,
3305,
351,
347,
349,
339,
342,
343,
357,
1045,
361,
333,
337,
336,
334,
332,
331,
344,
2257,
3231,
356,
858,
340,
330,
358,
2427,
353,
289,
297,
375,
296,
2273,
293,
292,
345,
354,
2973,
335,
290,
277,
291,
286,
359,
288,
271,
352,
276,
294,
3219,
283,
776,
281,
280,
279,
278,
295,
275,
270,
274,
285,
1625,
553,
272,
282,
1576,
284,
717,
418,
1869,
425,
537,
634,
1516,
463,
2859,
2191,
287,
273,
2177,
857,
1502,
269,
246,
954,
1055,
1976,
1445,
268,
2959,
227,
240,
234,
239,
329,
241,
245,
238,
243,
217,
1054,
235,
247,
231,
214,
224,
223,
1239,
215,
771,
197,
1134,
226,
216,
230,
194,
221,
195,
244,
208,
219,
3821,
237,
242,
233,
213,
236,
3422,
211,
207,
3905,
206,
222,
225,
210,
218,
3895,
232,
229,
2958,
196,
2592,
2415,
1703,
3990,
3509,
212,
220,
2294,
2858,
209,
193,
1234,
1429,
2327,
3218,
2426,
2141,
157,
2874,
168,
3230,
171,
164,
552,
169,
1044,
1575,
775,
172,
2176,
2272,
167,
137,
2707,
1624,
160,
146,
143,
161,
154,
162,
158,
144,
145,
147,
140,
165,
155,
142,
156,
228,
152,
121,
138,
135,
151,
119,
134,
166,
159,
129,
1444,
170,
163,
149,
148,
133,
120,
139,
124,
113,
150,
117,
112,
136,
141,
153,
1133,
115,
118,
89,
131,
123,
111,
110,
132,
93,
94,
128,
130,
96,
88,
99,
101,
122,
127,
126,
92,
90,
72,
106,
98,
105,
109,
125,
84,
107,
716,
69,
70,
424,
2780,
102,
267,
2113,
91,
108,
1752,
536,
3304,
417,
1868,
103,
1293,
3057,
2722,
87,
43,
42,
114,
462,
71,
41,
46,
45,
47,
953,
44,
33,
34,
39,
38,
97,
100,
29,
35,
36,
21,
83,
85,
30,
16,
28,
31,
11,
40,
19,
68,
20,
9,
22,
26,
37,
27,
322,
32,
4,
2,
8,
3,
14,
7,
1,
6,
941,
23,
25,
3072,
3904,
18,
3523,
1515,
1514,
192,
328,
1975,
5,
2957,
1765,
2501,
2972,
10,
633,
2112,
13,
1501,
15,
24,
2271,
774,
2140,
3894,
952,
2873,
2425,
2124,
205,
82,
95,
1132,
2580,
3820,
17,
86,
1574,
3662,
2255,
769,
3508,
2721,
632,
12,
2190,
2326,
856,
3421,
1043,
3056,
2293,
715,
1291,
1702,
416,
2779,
1623,
770,
3217,
3229,
847,
1851,
321,
1233,
3747,
2591,
204,
1751,
1428,
2971,
423,
551,
940,
1427,
461,
266,
3746,
855,
191,
2414,
1052,
4010,
1500,
535,
1053,
327,
460,
2111,
3893,
3903,
2292,
2424,
81,
2413,
846,
1974,
3071,
2856,
326,
2325,
1042,
773,
1750,
1701,
2720,
2872,
66,
3675,
2254,
2123,
2956,
939,
320,
550,
3302,
1290,
534,
1232,
1850,
1442,
714,
1443,
1573,
622,
631,
190,
415,
3055,
1513,
2270,
1973,
2855,
3892,
1426,
4008,
2871,
203,
951,
3660,
459,
2269,
2324,
2423,
116,
1865,
2719,
2412,
1051,
414,
1622,
3301,
1572,
630,
3419,
2704,
2253,
2122,
1849,
2268,
1848,
202,
413,
2173,
2291,
3829,
533,
3830,
2578,
3227,
1571,
3540,
325,
2955,
2121,
2109,
2954,
2854,
1440,
2422,
532,
2252,
2498,
458,
3053,
2703,
2870,
1698,
262,
853,
2775,
2108,
317,
412,
2267,
531,
2321,
842,
2409,
2107,
1697,
2852,
411,
2119,
1746,
410,
1912,
1955,
1918,
1910,
1915,
1916,
1882,
1911,
1996,
1901,
2304,
1907,
1880,
2017,
1903,
2982,
1864,
2590,
1900,
2215,
2512,
4002,
3914,
1895,
1862,
2134,
1899,
2336,
2435,
1904,
1826,
2789,
3849,
1834,
1863,
1830,
2867,
1811,
3083,
1832,
1839,
1824,
1828,
1809,
1823,
1819,
1818,
1821,
1812,
1836,
1827,
1835,
2135,
1810,
1813,
1816,
1806,
1788,
1829,
1902,
1804,
1817,
1825,
1798,
1803,
1805,
1785,
1814,
1799,
1942,
1781,
1786,
1800,
1822,
1801,
1787,
1782,
1778,
1763,
1807,
1783,
1796,
1779,
1815,
1820,
1784,
1794,
1780,
2511,
1762,
1777,
1802,
2998,
3765,
3533,
2450,
2601,
1719,
1720,
1738,
1731,
1721,
1726,
1727,
1716,
1808,
1713,
1729,
1861,
2200,
1712,
1722,
2883,
1718,
1717,
1730,
1723,
1733,
2303,
1724,
2185,
3248,
2981,
2281,
2016,
2521,
1877,
1776,
1761,
3913,
1995,
2716,
1793,
2472,
3066,
2600,
2335,
2264,
2434,
2967,
3239,
1715,
3932,
2133,
3933,
1985,
2589,
1994,
1714,
2616,
3331,
2788,
1686,
2302,
1672,
1685,
1725,
1711,
2510,
2280,
1678,
1688,
2866,
1663,
1680,
1682,
1675,
1670,
1665,
1662,
1687,
1660,
1683,
1677,
1666,
1684,
1671,
1653,
1679,
1681,
1645,
1651,
1647,
1648,
1654,
1637,
1659,
1649,
1664,
1668,
1673,
2980,
1635,
1634,
1646,
1661,
2149,
3430,
1643,
1667,
1612,
2199,
2449,
1636,
1655,
1652,
1611,
1633,
1608,
3096,
3238,
1658,
1609,
1650,
1644,
1584,
1610,
1595,
1606,
1598,
1860,
1604,
1602,
1642,
2799,
1599,
1601,
1591,
1893,
1760,
1607,
1605,
3330,
1594,
2184,
1676,
3998,
2715,
1592,
1876,
1775,
1984,
1600,
2997,
2966,
1593,
1792,
1585,
1632,
2279,
1590,
1550,
1551,
1559,
2881,
1560,
2198,
1561,
1597,
1544,
1641,
1547,
1539,
1542,
1558,
1536,
1583,
1553,
1545,
1562,
2730,
1548,
1557,
1554,
1511,
1540,
1552,
1541,
1546,
1543,
1512,
1534,
1538,
1529,
3912,
1510,
1532,
2132,
1528,
2787,
1525,
1526,
1527,
1549,
1533,
1531,
1524,
2714,
2509,
2996,
2865,
2334,
1983,
3685,
3329,
3841,
2448,
3064,
2729,
2148,
1530,
1523,
2263,
2979,
1710,
2470,
1483,
2183,
1537,
2433,
1478,
1485,
1482,
1479,
1535,
1481,
1468,
1859,
2301,
1460,
1457,
1759,
1487,
1458,
1484,
1456,
1474,
1473,
1486,
1464,
1475,
1459,
1453,
1476,
2798,
1469,
3429,
1480,
1454,
1455,
1462,
1466,
1477,
1467,
1472,
1465,
1993,
1791,
1774,
1522,
3095,
1509,
1437,
1875,
1451,
1461,
1463,
2333,
2147,
3931,
2131,
2599,
1452,
2432,
3237,
3997,
2965,
1404,
2519,
1436,
1992,
1402,
1631,
1470,
2786,
1387,
1405,
1709,
1409,
1356,
1400,
3840,
2197,
3930,
1410,
1351,
1408,
1344,
1395,
1399,
1345,
1406,
3911,
1407,
1401,
1412,
1403,
1371,
1389,
1302,
1390,
1359,
1393,
1338,
1284,
1392,
1347,
1411,
1283,
1391,
1280,
1285,
1397,
1301,
1323,
2880,
2447,
1288,
1758,
2587,
2864,
1336,
1388,
1398,
2978,
2300,
1355,
2182,
1982,
2278,
3996,
2728,
3063,
2469,
1281,
3236,
1394,
3296,
1216,
1396,
2713,
1219,
1196,
1159,
1213,
2964,
1588,
1210,
1197,
1215,
1214,
2598,
1170,
1157,
1144,
1158,
1171,
1147,
3529,
1142,
1212,
1150,
1145,
1148,
1209,
3762,
1149,
1154,
1630,
1143,
1640,
1450,
1508,
1211,
2508,
1139,
2332,
1773,
3079,
1858,
1153,
1218,
1337,
1300,
2181,
1217,
3838,
1435,
3078,
3910,
2797,
1708,
1991,
2863,
1097,
2431,
2130,
1151,
1141,
3062,
1152,
2586,
1069,
1067,
2262,
1083,
1065,
1146,
1092,
1089,
1087,
1064,
1068,
1085,
1088,
1093,
1066,
1521,
1063,
1071,
1096,
1076,
1075,
1061,
1077,
1094,
1279,
1082,
1080,
1086,
1079,
1074,
2196,
3683,
2507,
1095,
1060,
1070,
3295,
2277,
1072,
1078,
1091,
1081,
1471,
3235,
1062,
1090,
2446,
2963,
1981,
3327,
2146,
2261,
1299,
1140,
1073,
1639,
3995,
1059,
1789,
3309,
2977,
1507,
2712,
1757,
3837,
3761,
1449,
2597,
1084,
1138,
3760,
2299,
1990,
1013,
3061,
1021,
1011,
1019,
1434,
1014,
1756,
1520,
4017,
2430,
1020,
1629,
1010,
1015,
1005,
995,
1007,
997,
1012,
2785,
1022,
991,
1006,
1579,
994,
1008,
1009,
970,
1001,
971,
988,
972,
992,
993,
983,
986,
1018,
987,
1017,
989,
969,
984,
1003,
968,
998,
990,
996,
964,
981,
947,
946,
985,
3994,
1002,
980,
978,
999,
967,
962,
1000,
2585,
965,
975,
1004,
958,
963,
976,
982,
2596,
959,
979,
966,
1016,
3426,
2878,
2145,
1448,
2517,
2976,
961,
2962,
1269,
3513,
977,
918,
3326,
2711,
1980,
1707,
2195,
921,
3294,
960,
910,
893,
914,
922,
3909,
2260,
912,
907,
905,
3222,
908,
896,
916,
891,
909,
900,
887,
915,
872,
883,
895,
886,
892,
888,
903,
906,
904,
920,
881,
911,
913,
894,
1433,
899,
902,
885,
890,
869,
2331,
919,
884,
875,
882,
878,
865,
871,
867,
880,
863,
2795,
862,
889,
868,
2445,
879,
864,
876,
2467,
877,
945,
861,
870,
1628,
2506,
1058,
898,
901,
1771,
2726,
1506,
1518,
1137,
874,
812,
974,
1856,
866,
809,
1770,
815,
813,
3759,
957,
816,
2128,
2429,
817,
2275,
3908,
819,
3076,
796,
811,
794,
822,
818,
792,
1519,
793,
1298,
897,
805,
810,
791,
806,
821,
917,
1505,
790,
797,
802,
795,
785,
784,
781,
783,
789,
804,
787,
780,
799,
3425,
1706,
801,
803,
3681,
873,
2444,
820,
808,
782,
2330,
2466,
1979,
807,
2144,
788,
2961,
744,
798,
779,
746,
747,
800,
742,
2584,
731,
1755,
2975,
814,
786,
3546,
740,
733,
727,
1268,
720,
736,
730,
739,
745,
3221,
738,
741,
1047,
3526,
1447,
2794,
2783,
2194,
732,
729,
734,
2259,
1297,
2725,
2710,
737,
3233,
1871,
722,
3391,
4081,
3716,
2930,
2940,
3599,
3018,
2931,
4068,
3033,
3138,
2913,
3034,
3157,
3389,
3965,
3267,
3388,
2840,
2842,
4079,
3964,
3032,
2841,
2843,
3715,
4080,
2838,
2839,
3120,
2836,
3478,
3966,
2928,
2911,
2830,
3118,
3796,
3369,
3714,
3137,
3155,
3963,
2833,
2837,
2835,
3713,
3014,
3015,
3476,
2831,
2817,
3265,
3016,
2829,
3584,
2834,
2828,
3031,
2927,
3154,
3264,
3962,
2909,
2827,
2766,
2832,
2768,
4065,
3013,
2816,
3386,
2908,
3950,
3116,
3153,
3712,
2763,
2759,
3156,
3136,
2760,
3115,
2758,
2767,
2762,
2765,
2910,
3152,
4077,
2757,
3029,
2756,
2761,
2755,
3949,
3017,
3135,
2926,
3114,
3134,
2754,
3353,
4078,
3474,
3030,
3012,
3711,
2826,
2825,
3364,
2907,
3151,
2925,
2752,
2814,
2813,
2823,
3150,
2815,
3710,
3564,
3348,
2751,
4062,
3027,
2924,
3946,
3011,
3133,
2693,
3261,
2690,
3365,
3347,
2680,
2906,
2683,
2824,
3385,
2692,
2691,
2676,
2675,
2664,
2689,
3262,
2677,
2631,
2665,
2670,
2663,
2661,
3790,
2656,
2672,
2638,
2636,
2637,
2647,
2632,
2655,
2635,
2646,
2667,
2669,
2657,
2666,
2634,
2674,
2812,
3149,
2639,
2662,
2633,
2648,
3026,
2614,
2653,
2743,
2923,
2645,
2649,
2628,
2652,
3260,
2613,
3362,
2750,
2615,
2742,
2627,
2749,
3009,
2811,
3789,
3956,
3148,
2922,
3945,
2992,
2612,
2893,
4044,
2993,
3109,
3259,
2904,
3131,
2554,
2539,
2563,
2564,
3859,
3010,
3024,
2553,
2822,
2536,
2560,
2562,
2535,
3346,
2555,
3707,
2534,
2644,
3361,
2537,
2557,
2821,
2556,
2810,
2538,
2552,
3258,
2626,
2748,
2547,
3708,
2903,
2921,
2741,
3008,
2991,
2549,
2820,
2740,
3345,
3787,
2483,
3456,
3007,
2611,
3360,
2990,
3322,
2490,
2487,
2561,
3023,
2809,
2482,
3344,
2551,
2610,
3922,
2462,
2747,
2485,
3942,
3257,
2540,
2463,
2625,
2488,
2920,
2489,
2480,
2919,
2546,
2460,
2465,
2902,
3343,
2459,
2989,
2808,
3128,
3247,
2901,
2891,
3022,
2486,
3561,
3954,
4057,
3321,
2545,
3106,
3256,
2624,
3091,
3560,
2641,
2746,
3704,
2807,
3468,
2988,
2890,
3358,
3246,
2608,
4056,
2458,
3785,
3856,
3090,
2484,
2900,
2386,
2479,
2387,
2374,
3255,
2390,
2372,
2738,
3953,
2623,
2389,
2376,
3005,
2377,
3021,
2375,
2370,
2371,
2384,
2383,
2380,
2385,
2381,
2544,
2379,
3127,
2609,
2363,
2364,
2347,
2359,
2369,
2745,
2361,
2352,
2360,
2356,
2358,
2348,
2382,
2355,
2354,
2346,
2351,
3539,
3952,
3089,
3571,
3320,
2349,
2353,
2362,
2311,
2312,
2737,
2899,
2314,
3771,
2313,
2987,
3004,
3558,
3020,
3254,
2287,
3341,
2478,
2607,
2367,
2289,
3919,
3848,
3538,
2309,
2889,
3126,
2455,
2744,
2310,
2806,
3103,
2986,
3356,
3125,
3244,
2456,
2368,
2288,
3003,
2898,
2378,
3019,
2543,
2366,
2345,
3951,
4037,
2344,
2542,
2793,
2606,
2515,
3570,
2621,
2237,
3253,
2227,
2228,
2239,
2232,
2238,
2236,
2897,
2620,
3853,
2804,
2212,
2229,
2211,
2224,
2209,
2736,
2206,
3340,
2286,
2234,
2233,
2226,
2213,
2210,
2207,
2221,
2223,
2805,
2454,
2887,
3252,
2205,
2985,
2214,
3243,
3569,
2285,
2220,
3451,
2735,
2208,
2163,
2896,
2619,
2160,
2157,
2156,
3251,
2984,
2158,
2164,
4053,
2162,
3002,
2154,
3700,
3102,
2189,
3852,
2365,
2218,
2886,
2339,
2307,
2225,
3101,
2803,
3690,
2438,
2139,
2203,
4035,
2188,
2217,
3124,
3001,
2605,
2161,
2153,
2792,
3937,
2514,
2733,
2137,
3086,
3851,
2983,
2155,
2734,
3123,
2341,
2305,
2342,
3333,
3936,
3000,
3554,
2791,
3768,
2895,
3767,
2202,
2885,
2524,
2083,
3100,
2086,
2283,
3068,
2618,
2604,
2603,
2216,
2187,
3845,
2077,
2138,
2064,
2084,
2089,
2085,
2074,
2062,
3241,
2081,
2087,
2080,
2076,
2451,
2060,
2079,
2073,
3099,
2078,
2059,
2088,
2058,
2082,
2063,
2136,
2053,
2054,
2072,
2056,
2055,
2066,
2075,
2039,
2034,
2013,
2047,
2718,
2035,
2048,
2219,
2049,
2012,
2070,
2069,
2065,
2050,
2057,
2071,
2005,
2068,
2067,
2033,
2025,
2031,
2040,
2037,
2043,
2051,
2046,
2004,
2026,
2027,
2044,
2028,
2006,
1989,
2007,
2021,
2338,
2042,
2022,
2030,
2020,
2002,
2151,
2018,
2003,
2010,
1999,
1997,
3566,
2038,
2032,
1987,
2968,
2000,
2019,
2029,
2001,
2602,
2790,
2894,
1988,
2617,
2999,
3766,
2340,
2436,
2041,
1964,
3332,
2513,
2023,
2868,
3934,
2801,
2282,
3688,
2337,
2201,
2732,
1986,
1963,
1954,
1959,
1962,
1960,
1939,
1956,
2884,
3240,
3915,
1946,
1938,
3067,
1947,
2186,
1937,
1957,
1958,
1961,
1952,
1948,
1920,
1944,
1933,
1949,
1950,
1934,
1919,
1930,
1945,
3850,
1953,
2717,
1908,
1917,
1931,
1929,
1905,
1951,
1906,
1913,
1925,
4887,
4911,
4880,
4888,
4903,
4899,
4892,
4877,
4890,
4889,
4866,
4867,
4863,
4855,
4875,
4865,
4882,
4872,
4883,
4870,
4862,
4853,
4886,
4852,
4869,
4885,
4868,
4857,
4856,
4854,
4859,
4879,
4907,
4864,
4861,
4871,
4895,
4884,
4843,
4860,
4840,
4845,
4842,
4873,
4846,
4827,
4824,
4848,
4830,
4847,
4858,
4851,
4826,
4823,
4834,
4850,
4838,
4836,
4833,
4881,
4835,
4828,
4839,
4813,
4805,
4829,
4801,
4820,
4837,
4841,
4831,
4803,
4804,
4815,
4802,
4812,
4844,
4821,
4816,
4795,
4797,
4811,
4798,
4849,
4809,
4793,
4818,
4799,
4792,
4787,
4791,
4814,
4822,
4832,
4796,
4807,
4810,
4790,
4775,
4817,
4778,
4783,
4819,
4878,
4806,
4825,
4772,
4784,
4789,
4788,
4786,
4776,
4774,
4762,
4781,
4794,
4800,
4779,
4785,
4782,
4770,
4771,
4754,
4748,
4753,
4743,
4752,
4745,
4764,
4780,
4777,
4746,
4760,
4744,
4742,
4749,
4751,
4757,
4756,
4765,
4763,
4808,
4766,
4773,
4750,
4730,
4768,
4741,
4758,
4717,
4739,
4769,
4718,
4725,
4719,
4737,
4738,
4735,
4747,
4740,
4734,
4759,
4755,
4733,
4724,
4736,
4767,
4679,
4727,
4723,
4729,
4704,
4722,
4703,
4701,
4716,
4702,
4761,
4700,
4697,
4706,
4714,
4710,
4715,
4712,
4731,
4696,
4676,
4709,
4695,
4711,
4678,
4713,
4721,
4680,
4728,
4668,
4720,
4707,
4968,
4705,
4675,
4677,
4732,
4698,
4973,
4672,
4685,
4726,
4665,
4684,
4689,
4688,
4671,
4654,
4673,
4687,
4693,
4662,
4670,
4648,
4653,
4655,
4682,
4690,
4664,
4652,
4686,
4651,
4683,
4694,
4691,
4661,
4667,
4646,
4627,
4640,
4645,
4643,
4636,
4656,
4659,
4641,
4657,
4699,
4681,
4644,
4674,
4630,
4638,
4669,
4637,
4639,
4628,
4635,
4629,
4604,
4708,
4618,
4624,
4632,
4634,
4647,
4631,
4642,
4692,
4633,
4625,
4617,
4626,
4612,
4602,
4605,
4619,
4613,
4611,
4621,
4649,
4599,
4623,
4598,
4658,
4597,
4608,
4596,
4600,
4650,
4615,
4616,
4610,
4622,
4603,
4590,
4594,
4607,
4595,
4609,
4574,
4663,
4575,
4577,
4620,
4573,
4601,
4579,
4589,
4572,
4587,
4576,
4578,
4585,
4586,
4571,
4570,
4591,
4614,
4606,
4660,
4588,
4569,
4556,
4564,
4551,
4566,
4553,
4555,
4568,
4583,
4557,
4563,
4581,
4562,
4554,
4582,
4548,
4567,
4592,
4584,
4593,
4565,
4561,
4559,
4545,
4560,
4534,
4666,
4537,
4523,
4521,
4540,
4550,
4552,
4531,
4502,
4529,
4532,
4510,
4515,
4519,
4530,
4533,
4501,
4580,
4497,
4538,
4500,
4514,
4527,
4504,
4522,
4495,
4489,
4472,
4503,
4496,
4516,
4508,
4494,
4511,
4493,
4498,
4512,
4509,
4488,
4506,
4478,
4477,
4491,
4524,
4476,
4485,
4487,
4483,
4558,
4473,
4507,
4466,
4486,
4449,
4452,
4453,
4484,
4474,
4468,
4464,
4482,
4451,
4454,
4463,
4447,
4460,
4455,
4535,
4505,
4428,
4424,
4446,
4440,
4442,
4459,
4465,
4467,
4429,
4471,
4423,
4480,
4457,
4426,
4444,
4419,
4421,
4406,
4433,
4456,
4418,
4427,
4414,
4437,
4438,
4461,
4435,
4413,
4422,
4417,
4415,
4416,
4380,
4439,
4409,
4388,
4431,
4373,
4434,
4392,
4441,
4367,
4385,
4420,
4412,
4411,
4430,
4386,
4363,
4362,
4352,
4364,
4346,
4378,
4381,
4360,
4382,
4369,
4351,
4358,
4355,
4366,
4410,
4354,
4323,
4361,
4315,
4359,
4327,
4303,
4353,
4337,
4310,
4307,
4356,
4357,
4300,
4302,
4295,
4297,
4291,
4289,
4311,
4287,
4318,
4292,
4277,
4294,
4273,
4285,
4279,
4306,
4254,
4260,
4282,
4325,
4276,
4269,
4270,
4321,
4251,
4280,
4284,
4253,
4261,
4286,
4268,
4266,
4245,
4239,
4267,
4262,
4255,
4237,
4248,
4281,
4249,
4241,
4244,
4246,
4335,
4258,
4227,
4222,
4231,
4247,
4224,
4220,
4238,
4243,
4215,
4235,
4223,
4228,
4226,
4199,
4232,
4234,
4229,
4230,
4233,
4219,
4218,
4240,
4200,
4221,
4212,
4196,
4202,
4203,
4450,
4195,
4193,
4211,
4201,
4208,
4206,
4214,
4198,
4209,
4173,
4170,
4177,
4194,
4175,
4188,
4190,
4210,
4192,
4191,
4176,
4178,
4169,
4152,
4165,
4163,
4180,
4161,
4189,
4171,
4172,
4185,
4187,
4182,
4181,
4166,
4162,
4207,
4184,
4205,
4174,
4217,
4197,
4150,
4149,
4179,
4204,
4147,
4216,
4148,
4164,
4213,
4159,
4126,
4183,
4151,
4153,
4160,
4137,
4154,
4158,
3642,
4139,
4145,
4124,
4135,
4136,
4121,
4122,
4155,
4123,
4132,
3196,
4131,
3194,
4125,
4120,
4114,
4119,
4130,
3193,
4138,
4128,
4129,
3634,
3618,
4098,
3191,
3405,
3631,
3185,
4095,
3189,
3190,
3184,
3181,
4106,
3188,
4096,
3170,
3172,
3183,
3186,
4093,
3489,
4091,
3180,
4092,
4094,
3380,
3179,
3042,
3043,
3168,
3378,
3399,
3177,
3164,
3397,
3171,
3169,
4103,
3187,
3488,
3178,
3166,
3396,
3606,
3182,
3607,
3145,
3592,
3040,
3167,
3147,
3176,
3165,
3162,
3174,
3872,
3143,
3039,
3375,
3393,
3038,
3873,
3144,
3272,
3271,
2943,
3175,
3376,
2941,
3146,
4074,
3163,
4073,
3591,
2938,
3160,
3270,
2942,
3374,
3173,
3373,
2935,
3601,
3967,
3142,
2915,
2936,
2917,
3036,
2933,
2916,
3159,
3161,
3122,
3870,
3037,
2934,
4082,
3269,
2932,
3158,
3392,
3035,
3268,
3141,
3390,
2918,
3140,
2914,
5903,
5902,
5923,
5895,
5890,
5921,
5896,
5892,
5861,
5899,
5886,
5944,
5893,
5901,
5884,
5888,
5894,
5900,
5883,
5920,
5897,
5885,
5904,
5891,
5862,
5889,
5863,
5864,
5860,
5853,
5984,
5854,
5877,
5859,
5880,
5858,
5873,
5857,
5898,
5874,
5878,
5847,
5855,
5867,
5846,
5866,
5852,
5839,
5879,
5856,
5837,
5868,
5887,
5865,
5850,
5851,
5810,
5848,
5843,
5828,
5849,
5841,
5814,
5834,
5836,
5842,
5831,
5835,
5840,
5844,
5830,
5845,
5875,
5833,
5838,
5806,
5809,
5826,
5824,
5832,
5805,
5872,
5827,
5825,
5807,
5812,
5811,
5808,
5822,
5803,
5823,
5802,
5817,
5801,
5813,
5829,
5798,
5788,
5820,
5804,
5819,
5818,
5821,
5794,
5795,
5786,
5787,
5796,
5784,
5815,
5799,
5789,
5800,
5785,
5870,
5790,
5764,
5783,
5781,
5782,
5816,
5775,
5772,
5791,
5763,
5757,
5762,
5792,
5778,
5754,
5771,
5751,
5780,
5776,
5761,
5793,
5758,
5768,
5749,
5779,
5797,
5765,
5759,
5736,
5773,
5760,
5769,
5770,
5777,
5753,
5733,
5755,
5734,
5705,
5748,
5766,
5732,
5752,
5735,
5767,
5774,
5731,
5750,
5704,
5742,
5745,
5701,
5746,
5727,
5714,
5715,
5720,
5724,
5679,
5718,
5702,
5712,
5725,
5680,
5703,
5726,
5678,
5710,
5747,
5716,
5697,
5719,
5700,
5717,
5698,
5738,
5694,
5711,
5696,
5695,
5669,
5692,
5676,
5691,
5708,
5668,
5744,
5690,
5693,
5671,
5684,
5739,
5666,
5675,
5677,
5683,
5682,
5689,
5685,
5709,
5706,
5670,
5653,
5673,
5672,
5687,
5674,
5686,
5652,
5662,
5661,
5664,
5681,
5667,
5651,
5871,
5647,
5648,
5650,
5641,
5646,
5663,
5665,
5655,
5639,
5658,
5656,
5657,
5634,
5627,
5630,
5640,
5638,
5645,
5642,
5628,
5644,
5632,
5649,
5688,
5623,
5633,
5637,
5626,
5643,
5707,
5624,
5654,
5631,
5625,
5612,
5618,
5604,
5615,
5620,
5621,
5629,
5660,
5636,
5614,
5603,
5613,
5605,
5577,
5611,
5622,
5602,
5659,
5635,
5617,
5580,
5576,
5579,
5575,
5573,
5571,
5601,
5599,
5595,
5608,
5609,
5610,
5616,
5619,
5593,
5592,
5578,
5600,
5568,
5574,
5607,
5591,
5589,
5594,
5572,
5606,
5587,
5598,
5586,
5566,
5588,
5585,
5569,
5570,
5567,
5596,
5590,
5554,
5564,
5584,
5563,
5562,
5555,
5565,
5560,
5583,
5545,
5558,
5597,
5530,
5582,
5557,
5553,
5556,
5547,
5559,
5529,
5551,
5526,
5544,
5548,
5528,
5550,
5539,
5527,
5522,
5525,
5549,
5543,
5541,
5524,
5537,
5546,
5517,
5536,
5523,
5505,
5520,
5538,
5581,
5542,
5504,
5552,
5561,
5518,
5516,
5515,
5531,
5519,
5521,
5502,
5535,
5540,
5513,
5511,
5533,
5532,
5509,
5510,
5508,
5498,
5512,
5497,
5480,
5477,
5534,
5494,
5479,
5499,
5490,
5506,
5491,
5507,
5514,
5472,
5486,
5488,
5501,
5455,
5476,
5489,
5496,
5487,
5474,
5452,
5454,
5475,
5453,
5467,
5493,
5478,
5503,
5485,
5500,
5495,
5466,
5483,
5471,
5451,
5465,
5468,
5482,
5484,
5473,
5447,
5449,
5470,
5444,
5443,
5446,
5464,
5463,
5457,
5461,
5442,
5462,
5469,
5456,
5448,
5436,
5460,
5426,
5429,
5424,
5430,
5437,
5435,
5420,
5458,
5439,
5459,
5434,
5450,
5481,
5432,
5403,
5413,
5402,
5404,
5417,
5441,
5421,
5422,
5401,
5411,
5433,
5416,
5440,
5415,
5428,
5409,
5425,
5412,
5397,
5405,
5398,
5407,
5410,
5431,
5414,
5377,
5380,
5394,
5393,
5378,
5419,
5395,
5400,
5375,
5373,
5366,
5374,
5379,
5408,
5376,
5369,
5367,
5385,
5391,
5372,
5438,
5423,
5406,
5390,
5396,
5389,
5386,
5364,
5427,
5370,
5354,
5387,
5388,
5399,
5368,
5352,
5350,
5418,
5384,
5349,
5359,
5392,
5355,
5362,
5383,
5363,
5353,
5345,
5360,
5356,
5329,
5327,
5357,
5346,
5381,
5348,
5371,
5361,
5492,
5325,
5382,
5343,
5358,
5344,
5337,
5305,
5340,
5365,
5341,
5326,
5323,
5302,
5318,
5336,
5303,
5321,
5301,
5298,
5445,
5304,
5338,
5347,
5315,
5299,
5328,
5297,
5333,
5334,
5295,
5317,
5342,
5332,
5319,
5351,
5313,
5322,
5316,
5293,
5324,
5308,
5330,
5314,
5307,
5312,
5291,
5339,
5296,
5300,
5311,
5331,
5335,
5290,
5289,
5306,
5278,
5309,
5276,
5282,
5253,
5277,
5273,
5283,
5274,
5310,
5272,
5255,
5288,
5251,
5263,
5287,
5267,
5250,
5252,
5264,
5248,
5254,
5285,
5284,
5266,
5270,
5271,
5245,
5239,
5258,
5249,
5242,
5265,
5247,
5294,
5269,
5244,
5260,
5228,
5229,
5262,
5268,
5261,
5259,
5279,
5275,
5320,
5246,
5240,
5227,
5225,
5223,
5237,
5243,
5203,
5241,
5205,
5226,
5257,
5238,
5221,
5236,
5230,
5286,
5233,
5224,
5218,
5202,
5201,
5222,
5204,
5234,
5232,
5281,
5199,
5180,
5219,
5213,
5197,
5215,
5198,
5220,
5235,
5212,
5200,
5210,
5216,
5214,
5211,
5190,
5206,
5207,
5191,
5195,
5280,
5188,
5231,
5187,
5208,
5178,
5193,
5179,
5194,
5217,
5256,
5155,
5186,
5185,
5176,
5175,
5292,
5209,
5184,
5172,
5171,
5146,
5154,
5145,
5181,
5152,
5147,
5164,
5151,
5170,
5165,
5192,
5189,
5153,
5196,
5144,
5169,
5159,
5160,
5174,
5167,
5182,
5168,
5149,
5128,
5143,
5163,
5162,
5150,
5123,
5161,
5183,
5124,
5166,
5141,
5130,
5139,
5156,
5140,
5105,
5125,
5138,
5142,
5120,
5126,
5129,
5121,
5135,
5137,
5119,
5173,
5157,
5132,
5133,
5134,
5101,
5102,
5103,
5104,
5099,
5148,
5127,
5115,
5114,
5118,
5098,
5136,
5116,
5131,
5092,
5091,
5177,
5093,
5100,
5108,
5112,
5079,
5117,
5096,
5113,
5106,
5077,
5094,
5080,
5072,
5111,
5089,
5069,
5071,
5122,
5107,
5090,
5073,
5086,
5082,
5078,
5110,
5070,
5097,
5109,
5052,
5075,
5066,
5047,
5067,
5064,
5074,
5088,
5060,
5055,
5076,
5058,
5051,
5087,
5068,
5081,
5085,
5062,
5158,
5057,
5083,
5095,
5048,
5063,
5059,
5050,
5054,
5043,
5030,
5032,
5031,
5046,
5061,
5038,
5044,
5037,
5042,
5029,
5033,
5024,
5053,
5022,
5041,
5056,
5034,
5004,
5014,
5084,
5013,
5003,
5020,
5045,
5027,
5019,
5040,
5036,
5026,
5005,
4999,
5010,
4997,
5028,
5025,
5017,
5039,
5035,
5008,
5002,
5018,
4996,
4989,
4980,
4995,
5021,
4994,
5007,
4987,
4991,
5015,
4988,
5012,
4992,
5049,
5006,
4993,
5009,
4986,
4955,
4953,
4983,
5001,
4975,
4978,
4979,
4970,
5023,
5016,
4990,
4971,
5011,
4954,
4966,
4982,
4969,
4998,
4981,
4963,
4950,
5065,
4984,
4972,
4952,
4951,
4948,
4974,
4985,
4962,
4946,
4967,
4942,
4945,
4959,
5000,
4943,
4977,
4965,
4964,
4929,
4949,
4960,
4941,
4976,
4944,
4957,
4940,
4961,
4938,
4928,
4947,
4924,
4925,
4956,
4935,
4921,
4930,
4937,
4922,
4934,
4915,
4916,
4926,
4914,
4931,
4936,
4923,
4905,
4912,
4913,
4919,
4910,
4901,
4927,
4917,
4897,
4939,
4898,
4900,
4893,
4902,
4958,
4908,
4918,
4933,
4909,
4894,
4932,
4904,
4896,
4876,
4874,
4891,
4920,
768,
1441,
3070,
3902,
1749,
265,
2777,
845,
80,
1231,
1131,
3818,
65,
621,
2969,
2499,
3753,
549,
1499,
713,
1041,
1621,
767,
319,
264,
189,
1289,
79,
3069,
854,
64,
2323,
3817,
844,
938,
3987,
1620,
1425,
950,
1700,
620,
4004,
1699,
1972,
2577,
3752,
1129,
629,
1050,
766,
3891,
1121,
712,
1498,
1130,
188,
949,
201,
1040,
324,
3901,
1619,
937,
1229,
2576,
548,
2410,
2411,
843,
63,
1971,
2869,
2702,
3671,
711,
78,
619,
3285,
318,
1122,
2853,
3816,
1039,
852,
1439,
2421,
2322,
1424,
263,
3900,
2953,
3890,
765,
457,
323,
936,
77,
948,
1497,
1847,
2497,
1128,
3985,
1570,
187,
3899,
1423,
1970,
3213,
3889,
2420,
3815,
628,
1618,
3052,
200,
2170,
199,
456,
62,
1049,
1127,
3984,
2774,
627,
1119,
316,
2575,
935,
530,
1438,
1496,
3051,
2952,
764,
851,
618,
315,
186,
1038,
1037,
1569,
2250,
1495,
1048,
261,
2951,
710,
1422,
2320,
1969,
2574,
2249,
1126,
455,
626,
3983,
2700,
3223,
3224,
2118,
3888,
76,
2419,
1227,
1617,
2265,
185,
2408,
850,
198,
2169,
2106,
260,
934,
529,
2773,
1567,
1568,
3211,
3050,
61,
1226,
3414,
2573,
617,
763,
2319,
841,
1696,
2117,
409,
709,
3749,
708,
1695,
1036,
2105,
616,
2772,
314,
1421,
2850,
2699,
3049,
3501,
1968,
1125,
1225,
3887,
2418,
528,
2168,
2248,
3282,
1494,
2950,
933,
1616,
1228,
454,
2494,
625,
2407,
184,
1117,
259,
762,
2116,
2318,
59,
1844,
849,
1843,
848,
1120,
3886,
2849,
2417,
840,
74,
2572,
707,
2247,
932,
313,
615,
1566,
2493,
258,
3739,
2771,
3824,
1967,
1493,
2949,
1035,
67,
527,
2406,
408,
2167,
1224,
1420,
60,
183,
839,
1694,
1615,
2848,
624,
2697,
2698,
2104,
453,
2317,
3885,
1116,
2103,
2416,
1419,
1966,
2948,
3812,
761,
2166,
257,
1693,
931,
73,
623,
58,
1842,
1124,
2246,
452,
2770,
706,
1223,
2115,
3280,
1118,
3653,
312,
1742,
1034,
2492,
407,
1614,
1565,
1692,
406,
526,
3047,
311,
182,
256,
614,
838,
930,
3810,
2571,
181,
613,
2696,
760,
1564,
2491,
2570,
2245,
1418,
2847,
3046,
451,
3411,
1492,
1222,
2769,
3884,
1491,
2165,
525,
1417,
2102,
2405,
57,
1841,
1033,
837,
75,
612,
1032,
405,
1691,
705,
1740,
1221,
1115,
310,
2316,
759,
704,
2244,
1490,
1563,
2695,
450,
2101,
2315,
2404,
1613,
2403,
3651,
3278,
524,
836,
3206,
3883,
1965,
929,
1123,
2946,
2846,
2569,
758,
1416,
55,
1489,
3045,
2242,
611,
3652,
1739,
703,
3044,
1031,
2100,
3736,
3882,
404,
255,
3650,
179,
928,
309,
523,
1220,
449,
1690,
3881,
2402,
56,
2243,
2567,
702,
2945,
1112,
835,
54,
2844,
756,
2401,
757,
1113,
1111,
3734,
53,
1689,
308,
1415,
180,
2400,
178,
2099,
448,
834,
1488,
2241,
254,
403,
253,
3204,
2566,
402,
3494,
2399,
3880,
177,
252,
2240,
701,
1029,
3647,
755,
609,
610,
3203,
700,
925,
753,
833,
3274,
926,
1413,
2097,
307,
1114,
176,
306,
2098,
1110,
699,
832,
3804,
754,
3878,
1414,
401,
831,
400,
1030,
3201,
2096,
50,
608,
927,
2398,
305,
250,
52,
607,
2944,
1108,
399,
2095,
3273,
251,
751,
1027,
830,
3731,
606,
49,
3803,
398,
175,
3877,
173,
249,
605,
752,
304,
924,
829,
303,
698,
51,
3729,
1024,
3199,
3730,
2396,
3876,
750,
1109,
1028,
2093,
1107,
1023,
603,
2397,
1026,
604,
749,
2094,
1106,
3802,
2395,
1104,
2092,
48,
104,
3799,
601,
923,
826,
2394,
2091,
2392,
828,
748,
298,
300,
1025,
602,
302,
824,
3800,
3801,
248,
2393,
299,
598,
2090,
599,
2391,
600,
1100,
825,
301,
1103,
1105,
1102,
823,
827,
1098,
1101,
1099,
6087,
6084,
6094,
6078,
6089,
6080,
6082,
6091,
6102,
6086,
6079,
6068,
6075,
6054,
6099,
6100,
6077,
6081,
6088,
6064,
6085,
6074,
6056,
6062,
6058,
6065,
6069,
6083,
6072,
6070,
6076,
6073,
6071,
6090,
6060,
6098,
6055,
6059,
6063,
6050,
6067,
6095,
6052,
6066,
6051,
6061,
6025,
6057,
6045,
6046,
6037,
6093,
6026,
6041,
6049,
6044,
6027,
6097,
6034,
6048,
6031,
6096,
6032,
6023,
6047,
6040,
6042,
6030,
6022,
6038,
6036,
6033,
6035,
6043,
5972,
5995,
6006,
6021,
6019,
6016,
5993,
6029,
5971,
6001,
6028,
6018,
5974,
6024,
6000,
6008,
5970,
6009,
5996,
6017,
5998,
6012,
5992,
6014,
6015,
6053,
6002,
6039,
5994,
5975,
5987,
5942,
6004,
6010,
6007,
5986,
5973,
5983,
5967,
5989,
5941,
5961,
6020,
5988,
5969,
5997,
5985,
5978,
5991,
5939,
5999,
5943,
5965,
5981,
6005,
5937,
5957,
5953,
5955,
5951,
6011,
5954,
5976,
5946,
5918,
5948,
5932,
5915,
5945,
5936,
5916,
5947,
5931,
5934,
5982,
5966,
5913,
5949,
5952,
5980,
5940,
5938,
5950,
5924,
5925,
5927,
5910,
5926,
5922,
5933,
5914,
5911,
5917,
5907,
5908,
5906,
5909,
5919,
5905,
5929,
5935,
5930,
5928,
6003,
5912]

In [ ]:
riu = DataFrame(my_list, columns=['index_values'])

In [ ]:
h = pd.concat([riu, papers_df.text], axis=1)

In [ ]:
h.tail()

,index_values,text
4924,5935,"This is the post-print of Paxton, M. 2007. St..."
4925,5930,Mbulungeni Madiba University of Cape Town _...
4926,5928,"This article poses the question, who or what ..."
4927,6003,
4928,5912,This is the post-print of Hodgkinson-Williams...


In [ ]:
data.head(2)

,Unnamed: 0,URL,title,Author_,Date_,Description,Source_,Source_1,PDF_URL,PDF_text_
0,1,https://open.uct.ac.za//handle/11427/35468,A review of simulation models for the long-ter...,"Mukonda, Elton\n; \nCleary, Susan\n; \nLesosky...",06 Dec 2021,Abstract\r\n \r\n ...,BMC Health Services Research,Source\nBMC Health Services Research,https://open.uct.ac.za/bitstream/handle/11427/...,Mukonda˜\net˜al. BMC Health Services Research ...
1,2,https://open.uct.ac.za//handle/11427/35467,"Relationship, partner factors and stigma are a...","Gutin, Sarah A\n; \nHarper, Gary W\n; \nMoshas...",08 Dec 2021,Abstract\r\n \r\n ...,BMC Public Health,Source\nBMC Public Health,https://open.uct.ac.za/bitstream/handle/11427/...,Gutin˜et˜al. BMC Public Health (2021) ...


In [ ]:
hui = pd.merge(data, h, left_on='Unnamed: 0', right_on='index_values', how='inner')

In [ ]:
hui.shape

(4929, 12)

In [ ]:
hui.iloc[1490]

Unnamed: 0                                                   1678
URL                    https://open.uct.ac.za//handle/11427/29091
title              Occupational health: a world of false promises
Author_         LaDou, Joseph\n; \nLondon, Leslie\n; \nWatters...
Date_                                                 21 Nov 2018
Description     The response of the World Health Organization ...
Source_                                      Environmental Health
Source_1                             Source\nEnvironmental Health
PDF_URL         https://open.uct.ac.za/bitstream/handle/11427/...
PDF_text_       COMMENTARY\nOpenAccess\nOccupationalhealth:awo...
index_values                                                 1678
text             The world ’ s workforce sustains at least 370...
Name: 1490, dtype: object

In [ ]:
hui.URL.iloc[1490]

'https://open.uct.ac.za//handle/11427/29091'

In [ ]:
hui.head(5)

,Unnamed: 0,URL,title,Author_,Date_,Description,Source_,Source_1,PDF_URL,PDF_text_,index_values,text
0,1,https://open.uct.ac.za//handle/11427/35468,A review of simulation models for the long-ter...,"Mukonda, Elton\n; \nCleary, Susan\n; \nLesosky...",06 Dec 2021,Abstract\r\n \r\n ...,BMC Health Services Research,Source\nBMC Health Services Research,https://open.uct.ac.za/bitstream/handle/11427/...,Mukonda˜\net˜al. BMC Health Services Research ...,1,1. To identify model-based studies on T2D pop...
1,2,https://open.uct.ac.za//handle/11427/35467,"Relationship, partner factors and stigma are a...","Gutin, Sarah A\n; \nHarper, Gary W\n; \nMoshas...",08 Dec 2021,Abstract\r\n \r\n ...,BMC Public Health,Source\nBMC Public Health,https://open.uct.ac.za/bitstream/handle/11427/...,Gutin˜et˜al. BMC Public Health (2021) ...,2,Botswana has one of the highest burdens of HI...
2,3,https://open.uct.ac.za//handle/11427/35454,Analysis of Cameroon&rsquo;s Sectoral Policies...,"Tatah, Lambed\n; \nMapa-Tassou, Clarisse\n; \n...",02 Dec 2021,Physical inactivity is increasing in low- and ...,International Journal of Environmental Researc...,Source\nInternational Journal of Environmental...,https://open.uct.ac.za/bitstream/handle/11427/...,Article\nAnalysisofCameroon'sSectoralPolicieso...,3,Abstract: population. Keywords: physical act...
3,4,https://open.uct.ac.za//handle/11427/35453,The Exposome Approach in Allergies and Lung Di...,"López-Cervantes, Juan Pablo\n; \nLønnebotn, Ma...",01 Dec 2021,Emerging research suggests environmental expos...,International Journal of Environmental Researc...,Source\nInternational Journal of Environmental...,https://open.uct.ac.za/bitstream/handle/11427/...,Review\nTheExposomeApproachinAllergiesandLungD...,4,Abstract: become included in the exposome con...
4,5,https://open.uct.ac.za//handle/11427/35451,Towards an Understanding of the Effect of Addi...,"Gabriel, Sherlyn\n; \nvon Klemperer, Christoph...",23 Nov 2021,This paper presents insights into the blast re...,Materials,Source\nMaterials,https://open.uct.ac.za/bitstream/handle/11427/...,Article\nTowardsanUnderstandingoftheEffectofAd...,5,1. Through-thickness incident stress wave tra...


In [ ]:
hui.shape

(4929, 12)

In [ ]:
for col in hui.columns:
    if hui[col].dtype==object:
        hui[col]=hui[col].apply(lambda x: np.nan if x==np.nan else str(x).encode('utf-8', 'replace').decode('utf-8'))

In [ ]:
#saving the output
import pickle
hui.to_pickle('/content/gdrive/MyDrive/combined_data.pkl')
